# Returns From Irregular Time Series

Getting returns from an irregular time series should, for efficiency, always be done as a vector operation.  In **R** I prefer `data.table` with the `roll` and `mult` arguments.  In python, I use _pandas_ `reindex`.

Here we have in irregular time series of closing equity prices, with weekends missing of course.  From this, we form 2-day returns.

In [1]:
import quandl
import pandas as pd
import numpy as np
import datetime
import functools
%load_ext rpy2.ipython

In [5]:
@functools.lru_cache(maxsize=16)
def fetch_quandl(my_securities):
    qdata = quandl.get(list(my_securities), returns="pandas",
                      api_key=BrianBoonstraPrivateKeys['Quandl'])
    return qdata

In [6]:
daily = fetch_quandl(("EOD/HYLS",))['EOD/HYLS - Adj_Close'.format()]

In [7]:
daily.tail(11)

Date
2020-04-17    45.608678
2020-04-20    45.021399
2020-04-21    44.460000
2020-04-22    44.750000
2020-04-23    44.840000
2020-04-24    44.650000
2020-04-27    44.585000
2020-04-28    44.510000
2020-04-29    44.790000
2020-04-30    44.900000
2020-05-01    44.620000
Name: EOD/HYLS - Adj_Close, dtype: float64

In [8]:
later = daily.reindex(daily.index + pd.Timedelta('2d')).fillna(method='ffill')
later.index = daily.index
ret2 = later / daily - 1
returns_2_day = pd.DataFrame({'Adj_Close':daily, 'Later':later, 'TwoDayFwdReturn':ret2})
returns_2_day.tail(11).head(8)

,Adj_Close,Later,TwoDayFwdReturn
Date,,,
2020-04-17,45.608678,45.608678,0.000000
2020-04-20,45.021399,44.750000,-0.006028
2020-04-21,44.460000,44.840000,0.008547
2020-04-22,44.750000,44.650000,-0.002235
2020-04-23,44.840000,44.650000,-0.004237
2020-04-24,44.650000,44.650000,0.000000
2020-04-27,44.585000,44.790000,0.004598
2020-04-28,44.510000,44.900000,0.008762


In [9]:
%%R

Quandl::Quandl.api_key(BrianBoonstraPrivateKeys.Quandl)
r_daily = Quandl::Quandl(code="EOD/HYLS")

HYLS_P_tbl = data.table::data.table(r_daily[,c('Date', 'Adj_Close')], key='Date')
HYLS_P_tbl[, LDate := Date+2L]

HYLS_P_tbl[, Later := HYLS_P_tbl[.(Date=Date+2L), Adj_Close, roll=+2, mult="last", on="Date"]]
HYLS_P_tbl[, TwoDayFwdReturn := Later/Adj_Close-1]


head(tail(HYLS_P_tbl, 11) ,8)



         Date Adj_Close      LDate    Later TwoDayFwdReturn
1: 2020-04-17  45.60868 2020-04-19 45.60868     0.000000000
2: 2020-04-20  45.02140 2020-04-22 44.75000    -0.006028218
3: 2020-04-21  44.46000 2020-04-23 44.84000     0.008547009
4: 2020-04-22  44.75000 2020-04-24 44.65000    -0.002234637
5: 2020-04-23  44.84000 2020-04-25 44.65000    -0.004237288
6: 2020-04-24  44.65000 2020-04-26 44.65000     0.000000000
7: 2020-04-27  44.58500 2020-04-29 44.79000     0.004597959
8: 2020-04-28  44.51000 2020-04-30 44.90000     0.008762076
